## Luminosity comparison

In [1]:
from import_data import HMXB_parameters_Kaper
from import_data import HMXB_parameters
from import_data import supergiant_stellar_parameters
from import_data import falenga
from functions import scientific_notation
from functions import display_df
from import_data import stellar_params
from import_data import photometric_params
from import_data import BailerJones

from astropy.constants import R_sun, L_sun, sigma_sb

import pandas as pd
import dataframe_image as dfi
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as md
from tabulate import tabulate

Import data

In [2]:
# Import data
df_hmxb = HMXB_parameters()
df_sg_params = supergiant_stellar_parameters()
df_falenga = falenga()
df_stellar_params = stellar_params()
df_photometric_params = photometric_params()
df_BJ = BailerJones()
df_ID = pd.read_excel('tables/id_converter.xlsx')

df_BJ['source_id'] = df_BJ["source_id"].astype(np.int64)
df_ID['source_id'] = df_ID["source_id"].astype(np.int64)
df_BJ = pd.merge(df_ID, df_BJ, on='source_id')
df_BJ['source_id'] = df_BJ['source_id'].astype(str)

Make dataframes needed

In [3]:
df_L = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": [None] * len(df_hmxb['id']),
                              "L_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

### Calculate the luminosity from photometric data
This is the luminosity we observe

In [4]:
# for i in range(df_hmxb.shape[0]):
#     # Spectral type of object
#     spectral_type = df_hmxb.loc[i, "ST_short"]
#     # Expected (B-V) of object based on spectral type
#     BV0 = df_photometric_params[df_photometric_params['ST'] == spectral_type]["(B-V)0"].reset_index(drop=True).at[0]
#     # Observed (B-V) of object based on simbad filters
#     BVobs = df_hmxb.loc[i, '(B-V)obs']
#     # Bolometric correction (BC)
#     BCv = df_photometric_params[df_photometric_params['ST'] == spectral_type]["BCv"].reset_index(drop=True).at[0]

#     # Calculate extinction
#     Av = 3.2 * (BVobs - BV0)

#     # Calculate Absulute magnitude (visual)
#     Mv = df_hmxb.loc[i, 'mv'] - 5 * np.log10(df_hmxb.loc[i, 'distance']) + 5 - Av

#     # Calculate bolomatric absolute magnitude
#     Mbol = Mv + BCv

#     # Calculate the luminosity in solar luminosities
#     L = 10**((Mbol - 4.74) / (-2.5))

#     # Put L in luminosity dataframe
#     df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
#     # Put L in the dataframe
#     df_hmxb.loc[i, 'luminosity'] = L

<font color='red'>Using BJ distance</font>

In [5]:
for i in range(df_hmxb.shape[0]):
    # Object id
    id = df_hmxb.loc[i, 'id']
    # Spectral type of object
    spectral_type = df_hmxb.loc[i, "ST_short"]
    # Expected (B-V) of object based on spectral type
    BV0 = df_photometric_params[df_photometric_params['ST'] == spectral_type]["(B-V)0"].reset_index(drop=True).at[0]
    # Observed (B-V) of object based on simbad filters
    BVobs = df_hmxb.loc[i, '(B-V)obs']
    # Bolometric correction (BC)
    BCv = df_photometric_params[df_photometric_params['ST'] == spectral_type]["BCv"].reset_index(drop=True).at[0]
    # Distance
    distance = df_BJ.loc[df_BJ['id'] == id, 'r_med_geo'].reset_index(drop=True).at[0]

    # Calculate extinction
    Av = 3.2 * (BVobs - BV0)

    # Calculate Absulute magnitude (visual)
    Mv = df_hmxb.loc[i, 'mv'] - 5 * np.log10(distance) + 5 - Av

    # Calculate bolomatric absolute magnitude
    Mbol = Mv + BCv

    # Calculate the luminosity in solar luminosities
    L = 10**((Mbol - 4.74) / (-2.5))

    # Put L in luminosity dataframe
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
    # Put L in the dataframe
    df_hmxb.loc[i, 'luminosity'] = L

### Caclulate the luminosity from the radius and temperature of the optical star
This is the luminosity you would expect

In [6]:
for i in range(df_falenga.shape[0]):
    # Orbital separation
    a = df_falenga.loc[i, 'a'] * R_sun.value
    # Inclination
    inc = df_falenga.loc[i, 'i']
    # Semi eclipse angle
    theta_e = df_falenga.loc[i, 'semi_eclipse_angle']

    # Calculate Radius
    Rob = a * ((np.cos(np.deg2rad(inc)))**2 + (np.sin(np.deg2rad(inc)))**2 * (np.sin(np.deg2rad(theta_e)))**2)**(1/2)

    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["ST_short"].reset_index(drop=True).at[0]
    # Spectral temperature
    Teff = df_stellar_params[df_stellar_params["ST"] == spectral_type]["Teff"].reset_index(drop=True).at[0]

    # Calculate Luminosity in solar luminosities
    L = (4 * np.pi * Rob**2 * sigma_sb.value * Teff**4) / L_sun.value

    # Put L in luminosity dataframe
    df_L.loc[df_L["id"] == id, "L_expected"] = L
    df_L.loc[df_L["id"] == id, "ST_short"] = spectral_type
    df_L.loc[df_L["id"] == id, "ST"] = df_hmxb.loc[df_hmxb["id"] == id, "ST"]

<font color='yellow' size=5> Comparison of $L_{true}$ and $L_{expected}$(bolometric). </font>
- The $L_{expected}$ is calculated from $L = 4 \pi R^{2} \sigma T_{eff}^{4}$, where:
    * $R$ is calculated from the semi-eclipse ange $\theta_{e}$
    * $T_{eff}$ is taken from the models of Martin et al. (2005a)
- The $L_{true}$ is determined from photometric data from Simbad and $(B-V)_{0}$ is from martins et al. (2006)

Calculate $\frac{L_{true}}{L_{expected}}$ (all luminosities are in solar units)

In [7]:
# Filter out all rows with None values
df_L_filtered = df_L.copy().dropna()
df_L_filtered.reset_index(drop=True)

# Calculate fraction
df_L_filtered["L_true/L_expected"] = df_L_filtered["L_true"] / df_L_filtered["L_expected"]

Result

In [8]:
display_df(df_L_filtered)

| id        | $L_{true}[L_{\odot}]$   | $L_{expected}[L_{\odot}]$   | Spec. Type (short)   | Spec. Type   | $\frac{L_{true}}{L_{expected}}$   |
|:----------|:------------------------|:----------------------------|:---------------------|:-------------|:----------------------------------|
| SMC X-1   | 2.44E+04               | 2.30E+05                   | O9I                  | O9.7Ia+      | 1.06E-01                         |
| LMC X-4   | 1.78E+04               | 7.62E+04                   | O8III                | O8III        | 2.33E-01                         |
| Vela X-1  | 6.45E+05               | 7.88E+05                   | B0I                  | B0.5Ia       | 8.19E-01                         |
| Cen X-3   | 7.84E+06               | 1.36E+05                   | O9III                | O9III        | 5.78E+01                         |
| 4U1538-52 | 2.16E+05               | 8.70E+04                   | B0I                  | B0.2Ia       | 2.49E+00                         |
| 4U1700-37 | 4.96E+05               | 8.89E+05                   | O6I                  | O6Iafcp      | 5.57E-01                         |

<font color='yellow' size=5>Comparison of $L_{true}$ and $L_{expected}$(model martins et. al (2005a)). </font>

In [9]:
# Create new dataframe for the luminosities
df_L_model = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": df_L["L_true"].copy(),
                              "L_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

Get the luminosity for the object from the models of Martin et. al (2005a)

In [10]:
for i in range(df_hmxb.shape[0]):
    # Spectral type
    spectral_type = df_hmxb.loc[i, 'ST_short']
    # Luminosity from model
    logL = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "log(L/Lsun)"].reset_index(drop=True).at[0]
    L = 10**logL
    # Save luminosity in dataframe
    df_L_model.loc[i, "L_expected"] = L
    df_L_model.loc[i, "ST_short"] = spectral_type
    df_L_model.loc[i, "ST"] = df_hmxb.loc[i, "ST"]

In [11]:
# Calculate fraction
df_L_model["L_true/L_expected"] = df_L_model["L_true"] / df_L_model["L_expected"]

Result

In [12]:
display_df(df_L_model)

| id         | $L_{true}[L_{\odot}]$   | $L_{expected}[L_{\odot}]$   | Spec. Type (short)   | Spec. Type   | $\frac{L_{true}}{L_{expected}}$   |
|:-----------|:------------------------|:----------------------------|:---------------------|:-------------|:----------------------------------|
| 2S0114+650 | 1.59E+05               | 2.95E+05                   | B1I                  | B1Iae        | 5.40E-01                         |
| SMC X-1    | 2.44E+04               | 4.07E+05                   | O9I                  | O9.7Ia+      | 5.98E-02                         |
| LMC X-4    | 1.78E+04               | 2.24E+05                   | O8III                | O8III        | 7.93E-02                         |
| Vela X-1   | 6.45E+05               | 3.47E+05                   | B0I                  | B0.5Ia       | 1.86E+00                         |
| Cen X-3    | 7.84E+06               | 1.62E+05                   | O9III                | O9III        | 4.83E+01                         |
| GX301-2    | 5.37E+06               | 2.95E+05                   | B1I                  | B1.5Iaeq     | 1.82E+01                         |
| 4U1538-52  | 2.16E+05               | 3.47E+05                   | B0I                  | B0.2Ia       | 6.24E-01                         |
| 4U1700-37  | 4.96E+05               | 6.46E+05                   | O6I                  | O6Iafcp      | 7.68E-01                         |
| 4U1907+09  | 9.12E+05               | 4.79E+05                   | O8I                  | O8.5Iab      | 1.91E+00                         |
| LMC X-1    | 6.50E+04               | 4.79E+05                   | O8I                  | O8(f)p       | 1.36E-01                         |
| Cyg X-1    | 3.41E+05               | 4.07E+05                   | O9I                  | O9.7Iabpvar  | 8.38E-01                         |

<font color='yellow' size=5> Comparison of $R_{true}$ and $R_{expected}$ </font> <br></br>
where,
1. $R_{true}$ is calculated from the semi-eclipse angle.
2. $R_{expected}$ is based on the stellar parameters for the stars spectral type. Source Martin et. al (2005a). Get $L$ and $T_{eff}$ and calc R from $L = 4 \pi R^{2} \sigma T_{eff}^{4}$.

Both are calc. in terms of $R_{\odot}$

In [13]:
# Make radius dataframe
df_R = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "R_true": [None] * len(df_hmxb['id']),
                              "R_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

1. $R_{true}$

In [14]:
for i in range(df_falenga.shape[0]):
    # Orbital separation
    a = df_falenga.loc[i, 'a']
    # Inclination
    inc = df_falenga.loc[i, 'i']
    # Semi eclipse angle
    theta_e = df_falenga.loc[i, 'semi_eclipse_angle']

    # Calculate Radius
    Rob = a * ((np.cos(np.deg2rad(inc)))**2 + (np.sin(np.deg2rad(inc)))**2 * (np.sin(np.deg2rad(theta_e)))**2)**(1/2)

    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["ST_short"].reset_index(drop=True).at[0]

    # Put R in luminosity dataframe
    df_R.loc[df_R["id"] == id, "R_true"] = Rob
    df_R.loc[df_R["id"] == id, "ST_short"] = spectral_type
    df_R.loc[df_R["id"] == id, "ST"] = df_hmxb.loc[df_hmxb["id"] == id, "ST"]

2. $R_{expected}$

In [15]:
for i in range(df_hmxb.shape[0]):
    # Spectral type
    spectral_type = df_hmxb.loc[i, 'ST_short']

    # Luminosity from model
    logL = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "log(L/Lsun)"].reset_index(drop=True).at[0]
    L = 10**logL

    # Effective temperature from model
    Teff = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "Teff"].reset_index(drop=True).at[0]

    # Calculate the radius
    R = np.sqrt((L_sun.value / R_sun.value**2) * (L / (4 * np.pi * sigma_sb.value * Teff**4)))

    # Save luminosity in dataframe
    df_R.loc[i, "R_expected"] = R
    df_R.loc[i, "ST_short"] = spectral_type
    df_R.loc[i, "ST"] = df_hmxb.loc[i, "ST"]

In [16]:
# Calculate ratio
df_R["R_true/R_expected"] = df_R["R_true"] / df_R["R_expected"]

Result

In [17]:
df_R_filtered = df_R.copy()
df_R_filtered = df_R_filtered.dropna().reset_index(drop=True)

In [18]:
display_df(df_R_filtered)

| id        | $R_{true}[R_{\odot}]$   | $R_{expected}[R_{\odot}]$   | Spec. Type (short)   | Spec. Type   | $\frac{R_{true}}{R_{expected}}$   |
|:----------|:------------------------|:----------------------------|:---------------------|:-------------|:----------------------------------|
| SMC X-1   | 1.63E+01               | 2.16E+01                   | O9I                  | O9.7Ia+      | 7.52E-01                         |
| LMC X-4   | 7.98E+00               | 1.37E+01                   | O8III                | O8III        | 5.84E-01                         |
| Vela X-1  | 3.38E+01               | 2.25E+01                   | B0I                  | B0.5Ia       | 1.51E+00                         |
| Cen X-3   | 1.21E+01               | 1.32E+01                   | O9III                | O9III        | 9.14E-01                         |
| 4U1538-52 | 1.12E+01               | 2.25E+01                   | B0I                  | B0.2Ia       | 5.01E-01                         |
| 4U1700-37 | 2.32E+01               | 1.98E+01                   | O6I                  | O6Iafcp      | 1.17E+00                         |

<br></br>
### Compare distance from Bailer Jones et. al (2021) to distances found from the parallax from GAIA ($\frac{1}{p''}$)

In [19]:
pd.merge(df_hmxb, df_BJ, on='id')[['id', 'r_med_geo', 'r_med_photogeo', 'distance']]

,id,r_med_geo,r_med_photogeo,distance
0,2S0114+650,4475.7110,4527.0576,5091.204930
1,SMC X-1,16637.6640,15366.1540,250514.135002
2,LMC X-4,17214.2770,15435.0890,225469.097455
3,Vela X-1,1960.9159,1969.4784,2015.153991
4,Cen X-3,6784.6416,6752.4946,7208.804902
5,GX301-2,3604.5767,3546.0981,3990.086209
6,4U1538-52,5614.2314,5535.3643,7814.975864
7,4U1700-37,1499.6696,1513.9932,1580.438316
8,4U1907+09,3571.8506,3170.2651,4298.703408
9,LMC X-1,23756.2090,21067.4060,36402.117341
